# Laboratorio 04 - COVID

## Carga de datos csv en spark desde un bucket S3.

In [ ]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('covid_lab').getOrCreate()
data=spark.read.csv('s3://acanomdatasets/covid/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)

In [ ]:
data.printSchema()

## borrar y crear algunas columnas
- Columnas borradas: Codigo DIVIPOLA, fecha reporte web
- Columnas creadas: Días entre notificación y diagnostico, y Menor de 18

### Borrrar columnas

In [ ]:
data2=data.drop('Codigo DIVIPOLA', 'fecha reporte web')
data2.printSchema()

### Crear columnas

In [ ]:
# días entre fecha de notificación (SIVIGILA) y fecha de diagnostico.
data2 = data2.withColumn(
    "Días entre notificación y diagnostico", 
    (data2["Fecha diagnostico"].cast("long") - data2["Fecha de notificación"].cast("long"))/86400
)
data2.select("Fecha de notificación", "Fecha diagnostico", "Días entre notificación y diagnostico").show(10)
data2.select("Fecha de notificación", "Fecha diagnostico", "Días entre notificación y diagnostico").count()

In [ ]:
# Menores de 18 años
data2 = data2.withColumn("Menor de 18", data2["Edad"] < 18)
data2.select("Edad", "Menor de 18").filter(data2["Menor de 18"]==True).show(10)
data2.select("Edad", "Menor de 18").filter(data2["Menor de 18"]==True).count()

In [ ]:
data2.printSchema()

## Realizar filtrados de datos por alguna información que le parezca interesante
- Filtro 1: Personas en Antioquia con el virus importado
- Filtro 2: Menores de 18 años fallecidos
- Filtro 3: Personas en Antioquia recuperados

In [ ]:
filtro1 = data2.filter(
   (data2["Departamento o Distrito "] == "Antioquia") & (data2["Tipo"] == "Importado")
).select(
    "Ciudad de ubicación", "Edad", "Sexo",  "Tipo", "Estado"
).orderBy("Edad")
filtro1.show()
filtro1.count()

In [ ]:
filtro2 = data2.filter(
   ((data2["Menor de 18"] == True) & (data2["Estado"]=="Fallecido"))
).select(
    "Ciudad de ubicación", "Edad", "Menor de 18", "Sexo",  "Tipo", "Estado"
).orderBy("Edad")
filtro2.show()
filtro2.count()

In [ ]:
filtro3 = data2.filter(
   ((data2["Departamento o Distrito "] == "Antioquia") & (data2["atención"]=="Recuperado"))
).select(
    "Ciudad de ubicación", "Edad", "Menor de 18", "Sexo", "atención"
).orderBy("Edad")
filtro3.show()
filtro3.count()

## Realizar alguna agrupación y consulta de datos categorica, por ejemplo número de casos por región o por sexo/genero.
- Grupo 1: Personas agrupadas por país de procedencia Fallecidos
- Grupo 2: Personas agrupadas por Ciudad de ubicación en el departamento de Antioquia con su tipo de atención

In [ ]:
grupo1 = data2.filter(data2["atención"] == "Fallecido").select(
    "atención", "estado", "País de procedencia"
).groupBy('País de procedencia', "atención").count().orderBy('count', ascending=False)
grupo1.show()

In [ ]:
grupo2 = data2.filter(data2["Departamento o Distrito "] == "Antioquia").select(
    "atención", 'Ciudad de ubicación', 'Departamento o Distrito '
).groupBy('Ciudad de ubicación', "atención").count().orderBy('count', ascending=False)
grupo2.show()

## Grave los resultados en un bucket público en S3
Bucket: s3://acanomoutputs/covid

In [ ]:
url = "s3://acanomoutputs/covid"

data2.coalesce(1).write.format("csv").option("header","true").save(url + "/data2")
filtro1.coalesce(1).write.format("csv").option("header","true").save(url + "/filtro1")
filtro2.coalesce(1).write.format("csv").option("header","true").save(url + "/filter2")
filtro3.coalesce(1).write.format("csv").option("header","true").save(url + "/filter3")
grupo1.coalesce(1).write.format("csv").option("header","true").save(url + "/grupo1")
grupo2.coalesce(1).write.format("csv").option("header","true").save(url + "/grupo2")

# Conclusiones

Se observa que hay un gran número de personas en Antioquia (123) que importaron el virus de otros paises. 192 personas se han recuperado en Antioquia. Han muerto 3 menores de edad en Colombia. Por otra parte, basados en los paises de donde se ha importado el virus a Colombia, Estados Unidos y España presentan el mayor número de muertes. Medellín y Bello lideran el mayor número de casos en Antioquia.